In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import spacy
from wordcloud import WordCloud

In [13]:
data = pd.read_csv('use_df2.csv')
df= data.copy()
df. head()

,Unnamed: 0,Title,Series or Movie_x,COUNTRY,Release Date_x,GENRE,VALUE,Genre,Tags,Languages,...,Rotten Tomatoes Score,Metacritic Score,Awards Received,Awards Nominated For,Boxoffice,Release Date_y,Netflix Release Date,Production House,Summary,IMDb Votes
0,0,The Queens Gambit,TV,United States,2020.10.23,Drama,44867,"Biography, Drama, History","TV Dramas,US TV Shows,TV Shows Based on Books,...","English, German, French",...,96.0,91.0,96.0,97.0,"$56,441,711",2006.11.17,2020.10.23,"Pathé Pictures, Scott Rudin Productions","In a 1950s orphanage, a young girl reveals an ...",104495.0
1,1,365 Days,Movie,Poland,2020.6.7,Romance,42149,"Drama, Romance","Polish Movies,Polish Dramas,Dramas,Romantic Dr...","Polish, English, Italian",...,NaN,NaN,NaN,NaN,NaN,2020.6.7,2020.4.2,NaN,A fiery executive in a spiritless relationship...,50125.0
2,2,Emily in Paris,TV,United States,2020.10.2,Drama,27138,"Comedy, Drama, Romance","Comedy Programmes,Drama Programmes,Romantic TV...","French, English",...,NaN,NaN,1.0,3.0,NaN,2020.10.2,2020.10.2,NaN,"After landing her dream job in Paris, Chicago ...",45000.0
3,3,Lucifer,TV,United States,2016.1.25,Superhero,25729,"Horror, Thriller","TV Dramas,Police TV Shows,TV Sci-Fi & Fantasy,...",English,...,NaN,NaN,NaN,NaN,NaN,NaN,2017.7.24,NaN,"Bored with being the Lord of Hell, the devil r...",57.0
4,4,The Umbrella Academy,TV,United States,2019.2.15,Superhero,23480,"Action, Adventure, Comedy, Drama, Fantasy, Sci-Fi","TV Sci-Fi & Fantasy,US TV Shows,TV Action & Ad...",English,...,NaN,NaN,7.0,28.0,NaN,2019.2.15,2019.2.15,NaN,"Reunited by their fathers death, estranged sib...",159677.0


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2913 entries, 0 to 2912
Data columns (total 29 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             2913 non-null   int64  
 1   Title                  2913 non-null   object 
 2   Series or Movie_x      2913 non-null   object 
 3   COUNTRY                2727 non-null   object 
 4   Release Date_x         2851 non-null   object 
 5   GENRE                  2549 non-null   object 
 6   VALUE                  2913 non-null   int64  
 7   Genre                  2824 non-null   object 
 8   Tags                   2904 non-null   object 
 9   Languages              2796 non-null   object 
 10  Series or Movie_y      2913 non-null   object 
 11  Hidden Gem Score       2786 non-null   float64
 12  Country Availability   2909 non-null   object 
 13  Runtime                2913 non-null   object 
 14  Director               2243 non-null   object 
 15  Writ

In [15]:
df.isnull().sum()

Unnamed: 0                  0
Title                       0
Series or Movie_x           0
COUNTRY                   186
Release Date_x             62
GENRE                     364
VALUE                       0
Genre                      89
Tags                        9
Languages                 117
Series or Movie_y           0
Hidden Gem Score          127
Country Availability        4
Runtime                     0
Director                  670
Writer                    472
Actors                    107
View Rating               652
IMDb Score                127
Rotten Tomatoes Score    1317
Metacritic Score         1438
Awards Received          1407
Awards Nominated For     1026
Boxoffice                1593
Release Date_y            127
Netflix Release Date        0
Production House         1419
Summary                     2
IMDb Votes                127
dtype: int64

In [16]:
df = df.drop(columns = ['Rotten Tomatoes Score', 'Metacritic Score', 'Awards Received','Awards Nominated For','Boxoffice', 'Production House'])

In [17]:
df_country=df[['Title','VALUE', 'COUNTRY','GENRE', 'Country Availability','Languages']]

In [18]:
df_country.head(1)

,Title,VALUE,COUNTRY,GENRE,Country Availability,Languages
0,The Queens Gambit,44867,United States,Drama,"Lithuania,Canada,Australia,Poland,Mexico,Franc...","English, German, French"


In [19]:
#pd.get_dummies(df_country)

In [20]:
df_country.set_index('Title',inplace=True)

In [21]:
df_country.head(2)

,VALUE,COUNTRY,GENRE,Country Availability,Languages
Title,,,,,
The Queens Gambit,44867,United States,Drama,"Lithuania,Canada,Australia,Poland,Mexico,Franc...","English, German, French"
365 Days,42149,Poland,Romance,"Poland,Russia,United Kingdom,Slovakia,Canada,G...","Polish, English, Italian"


In [22]:
result1 = df_country['GENRE'].str.split(',')  # 리스트화
result1 = result1.apply(lambda x: pd.Series(x))
result1.stack()
result1.stack().reset_index(level=1, drop=True)

Title
The Queens Gambit                 Drama
365 Days                        Romance
Emily in Paris                    Drama
Lucifer                       Superhero
The Umbrella Academy          Superhero
                             ...       
The Bourne Supremacy             Action
Just Friends                    Romance
Child 44                       Thriller
Dracula Untold                   Horror
Riddick                 Science Fiction
Length: 2549, dtype: object

In [23]:
result1 = result1.stack().reset_index(level=1, drop=True).to_frame('Genre_single')
result1

,Genre_single
Title,
The Queens Gambit,Drama
365 Days,Romance
Emily in Paris,Drama
Lucifer,Superhero
The Umbrella Academy,Superhero
...,...
The Bourne Supremacy,Action
Just Friends,Romance
Child 44,Thriller


In [24]:
rs = result1.reset_index()
rs.head(5)

,Title,Genre_single
0,The Queens Gambit,Drama
1,365 Days,Romance
2,Emily in Paris,Drama
3,Lucifer,Superhero
4,The Umbrella Academy,Superhero


In [25]:
df_country1=df_country.reset_index()

In [26]:
df_country1.head(1)

,Title,VALUE,COUNTRY,GENRE,Country Availability,Languages
0,The Queens Gambit,44867,United States,Drama,"Lithuania,Canada,Australia,Poland,Mexico,Franc...","English, German, French"


In [27]:
result2=pd.merge(df_country,result1, on = 'Title')

In [28]:
result2

,VALUE,COUNTRY,GENRE,Country Availability,Languages,Genre_single
Title,,,,,,
The Queens Gambit,44867,United States,Drama,"Lithuania,Canada,Australia,Poland,Mexico,Franc...","English, German, French",Drama
365 Days,42149,Poland,Romance,"Poland,Russia,United Kingdom,Slovakia,Canada,G...","Polish, English, Italian",Romance
Emily in Paris,27138,United States,Drama,"Canada,Lithuania,Australia,Poland,Mexico,Franc...","French, English",Drama
Lucifer,25729,United States,Superhero,"South Korea,Iceland,Hungary,Australia,Hong Kon...",English,Superhero
The Umbrella Academy,23480,United States,Superhero,"Lithuania,Brazil,Israel,Spain,Slovakia,United ...",English,Superhero
...,...,...,...,...,...,...
The Bourne Supremacy,1,United States,Action,"Japan,France,Russia,South Korea,Hungary,Belgiu...","English, Russian, German, Italian",Action
Just Friends,1,United States,Romance,"Romania,South Korea,United Kingdom,Canada,Czec...",English,Romance
Child 44,1,United States,Thriller,"Germany,India","English, Russian",Thriller


In [29]:
rs2_reset_index=result2.reset_index()

In [30]:
class_group=rs2_reset_index.groupby(['Genre_single','COUNTRY'])
class_group

In [31]:
rs2_reset_index

,Title,VALUE,COUNTRY,GENRE,Country Availability,Languages,Genre_single
0,The Queens Gambit,44867,United States,Drama,"Lithuania,Canada,Australia,Poland,Mexico,Franc...","English, German, French",Drama
1,365 Days,42149,Poland,Romance,"Poland,Russia,United Kingdom,Slovakia,Canada,G...","Polish, English, Italian",Romance
2,Emily in Paris,27138,United States,Drama,"Canada,Lithuania,Australia,Poland,Mexico,Franc...","French, English",Drama
3,Lucifer,25729,United States,Superhero,"South Korea,Iceland,Hungary,Australia,Hong Kon...",English,Superhero
4,The Umbrella Academy,23480,United States,Superhero,"Lithuania,Brazil,Israel,Spain,Slovakia,United ...",English,Superhero
...,...,...,...,...,...,...,...
3247,The Bourne Supremacy,1,United States,Action,"Japan,France,Russia,South Korea,Hungary,Belgiu...","English, Russian, German, Italian",Action
3248,Just Friends,1,United States,Romance,"Romania,South Korea,United Kingdom,Canada,Czec...",English,Romance
3249,Child 44,1,United States,Thriller,"Germany,India","English, Russian",Thriller
3250,Dracula Untold,1,United States,Horror,"Switzerland,Canada,Poland,Italy,Greece,Czech R...","English, Turkish",Horror


In [32]:
class_group.groups

{('Action', 'Angola'): [145], ('Action', 'Bulgaria'): [1496], ('Action', 'Canada'): [571, 572, 575, 576, 1143, 1215, 1494, 1960, 1998, 2009, 2219], ('Action', 'China'): [579, 580, 583, 584, 2896, 3224], ('Action', 'France'): [303, 554, 1741, 2008, 2010, 2087, 2248, 2390, 2396, 2551, 2552, 2553, 2554, 2649], ('Action', 'Hong-Kong'): [284, 565, 1339, 1664, 2831, 2834, 2988, 3162, 3163, 3164, 3165], ('Action', 'India'): [705, 1653, 1654, 1657, 1658, 1800, 2353, 3049, 3217], ('Action', 'Israel'): [803], ('Action', 'Japan'): [1073, 1827], ('Action', 'Malaysia'): [2155], ('Action', 'Mexico'): [192], ('Action', 'Nigeria'): [1466, 1467, 1470, 1471, 2496], ('Action', 'South Korea'): [1453, 1487, 1832, 1910, 2114, 2121, 3126], ('Action', 'Spain'): [2344], ('Action', 'Thailand'): [1314, 2286], ('Action', 'United Kingdom'): [724, 1307, 1419, 1601, 1645, 1646, 1649, 1650, 2280, 2544, 2786, 2952, 3077], ('Action', 'United States'): [25, 30, 31, 32, 33, 34, 35, 36, 37, 38, 98, 113, 115, 209, 216, 241

In [33]:
class_group.count().head(30)

Title  VALUE  GENRE  Country Availability  \
Genre_single COUNTRY                                                     
Action       Angola              1      1      1                     1   
             Bulgaria            1      1      1                     1   
             Canada             11     11     11                    11   
             China               6      6      6                     6   
             France             14     14     14                    14   
             Hong-Kong          11     11     11                    11   
             India               9      9      9                     9   
             Israel              1      1      1                     1   
             Japan               2      2      2                     2   
             Malaysia            1      1      1                     1   
             Mexico              1      1      1                     1   
             Nigeria             5      5      5                     5   
             South Korea         7      7      7                     7   
             Spain               1      1      1                     1   
             Thailand            2      2      2                     2   
             United Kingdom     13     13     13                    13   
             United States     201    201    197                   201   
Adventure    Germany             2      2      2                     2   
             India               1      1      1                     1   
             Norway              1      1      1                     1   
             Switzerland         1      1      1                     1   
             United Kingdom     11     11     11                    11   
             United States      74     74     74                    74   
Animated     Canada             13     13     13                    13   
             China               6      6      6                     6   
             Denmark             1      1      1                     1   
             France              2      2      2                     2   
             Germany             3      3      3                     3   
             Japan              19     19     19                    19   
             Philippines         2      2      2                     2   

                             Languages  
Genre_single COUNTRY                    
Action       Angola                  1  
             Bulgaria                1  
             Canada                 11  
             China                   6  
             France                 14  
             Hong-Kong              11  
             India                   9  
             Israel                  1  
             Japan                   1  
             Malaysia                1  
             Mexico                  1  
             Nigeria                 5  
             South Korea             7  
             Spain                   1  
             Thailand                2  
             United Kingdom         13  
             United States         200  
Adventure    Germany                 2  
             India                   1  
             Norway                  1  
             Switzerland             1  
             United Kingdom         11  
             United States          74  
Animated     Canada                 12  
             China                   6  
             Denmark                 1  
             France                  2  
             Germany                 2  
             Japan                  18  
             Philippines             2

In [34]:
class_group.mean()

VALUE
Genre_single COUNTRY                    
Action       Angola          3239.000000
             Bulgaria         128.000000
             Canada           306.000000
             China            245.166667
             France           201.071429
...                                  ...
War          United Kingdom   226.200000
             United States    720.461538
Western      Denmark           36.000000
             France            94.000000
             United States     54.692308

[310 rows x 1 columns]

In [35]:
#df_country.groupby(['COUNTRY','').mean()

In [36]:
class_group.mean().index
    

MultiIndex([(  'Action',         'Angola'),
            (  'Action',       'Bulgaria'),
            (  'Action',         'Canada'),
            (  'Action',          'China'),
            (  'Action',         'France'),
            (  'Action',      'Hong-Kong'),
            (  'Action',          'India'),
            (  'Action',         'Israel'),
            (  'Action',          'Japan'),
            (  'Action',       'Malaysia'),
            ...
            ('Thriller',  'United States'),
            (     'War',          'Egypt'),
            (     'War',        'Hungary'),
            (     'War',          'India'),
            (     'War',         'Israel'),
            (     'War', 'United Kingdom'),
            (     'War',  'United States'),
            ( 'Western',        'Denmark'),
            ( 'Western',         'France'),
            ( 'Western',  'United States')],
           names=['Genre_single', 'COUNTRY'], length=310)